# Case Statement

Here is a simple query using a simple group by:

In [25]:
%%saql
q = load "opportunities";
q = group q by ('Type');
q = foreach q generate 'Type', sum('Amount') as 'sum_Amount';

User "kim@waverocks.de" running query on https://na30.salesforce.com


,Type,sum_Amount
0,Existing Business,164608134.0
1,New Business,374465875.0
2,New Business / Add-on,266316639.0


You might want to bucket the result on the fly. For example, how could you group both `New Business` and `New Business / Add-on` as `New` and rename `Existing Business` as `Old Business`? That's when you need to use the `case` statement!

## Binning

Binning is a technique used to group values (either dimensions or measures) into a smaller number of “bins”. In the following example, we bucket all records that have Type equals to “Services” or has word “New” as “New”. All the records that have word “Business” in the Type as “Old” and all other types are bucketed as “Other”. When this query executes, we get a new dimension called Type_Bucket that does not exist in the source dataset. Also, note that the order of the conditions is important. For example, the Type value “New Business” will be bucketed as “New Business” and not “Old Business” because the condition 'Type' matches "New "occurs before 'Type' matches "Business".

In [24]:
%%saql
q = load "opportunities";
q = foreach q generate (case
        when 'Type' matches "New" or 'Type' == "Services" then "New"
        when 'Type' matches "Business" then "Old Business"
        else "Other"
    end) as 'Type_Bucket', 'Amount';
q = group q by ('Type_Bucket');
q = foreach q generate 'Type_Bucket', sum('Amount') as 'sum_Amount';

User "kim@waverocks.de" running query on https://na30.salesforce.com


,Type_Bucket,sum_Amount
0,New,640782514.0
1,Old Business,164608134.0


## Threshold

In this example, a visual indication is given when a threshold value is reached using the traffic light metaphor. The SAQL shown backs a donut chart with a grouped by dimension ‘RAG’ with various values Red, Amber, Green, which indicates the threshold segmentation reached. An XMD entry is used to assign the color to the dimension value. The key performance indicators are Quota Attainment which correspond to the Total Percentage Achieved of revenue against the target; the SAQL makes use of those KPI steps when calculating the thresholds.

**Source step:** 

```json
"TotalActual__c_7": {
    "selectMode": "single",
    "query": {
        "measures": [
            [
                "sum",
                "TotalActual__c"
            ]
        ]
    },
    "datasets":[
        { "name": "Sales_Target" }
    ],
    "visualizationParameters": {
        "visualizationType": "hbar"
    },
    "isGlobal": false,
    "start": null,
    "useGlobal": true,
    "type": "aggregate",
    "isFacet": true
}
```

And here is the SAQL query (the quota is hardcoded here but you could pull it from another dataset):

In [31]:
%%saql
q = load "opportunities";
q = group q by all;
q = foreach q generate (sum('Amount') / 900000000) * 100 as 'quota_Attainment';
q =foreach q generate 'quota_Attainment',
    (case
        when 'quota_Attainment' >= 100 then "Green"
        when 'quota_Attainment' < 100 && 'quota_Attainment' > 60 then "Amber"
        else "Red"
    end ) as 'RAG';

User "kim@waverocks.de" running query on https://na30.salesforce.com


,RAG,quota_Attainment
0,Amber,89.48785


The following needs to be added to the XMD file to set threshold colors created in the above
query.

```json
"colors": {
    "RAG": {
        "Red": "#FF3030",
        "Green": "#008B00",
        "Amber": "#FFD700"
    }
}
```

Here is how it would look:

![Salesforce Overview](/images/case/dashboard-example.jpg)

[>> Time-based Analysis](/notebooks/time-based-analysis)